In [1]:
from pylab import *
import pandas as pd
from ase import Atoms, units
from ase.build import add_adsorbate, molecule
from ase.visualize import view
from ase.constraints import FixAtoms
from ase.io import write as wrt
from ase.io import espresso
import nglview
import os, sys
sys.path.append(os.path.abspath('../function_files'))

from output_analysis import energy, force, ase_out, bond_length, save_images
from write_in import QE_input
from vibration_analysis import scf_input, scf_out, ph_input, ph_out
from ase.io import read as rd
from ase.thermochemistry import IdealGasThermo, HarmonicThermo

## Define FF on Pt from optimized structure - atop 2

In [2]:
pre = 'FFPt_ph'

In [3]:
FFPt = rd('../Preferred Configurations/Pt111/FF_species/FF_Pt111.xyz', index = -1)

In [4]:
path = '../data/QE_files/'

# SCF Calculation

## write input file for initial scf with tighter convergence

In [5]:
const = FixAtoms(indices = arange(0,16))
FFPt.set_constraint(const)

In [6]:
scf_input(path+pre+'_scf.in', FFPt, pre+'_scf', 'ads')

## SCF Energy Analysis

In [7]:
energy = scf_out(path+pre+'_scf'+'.out')

In [8]:
energy

-44460.286570178054

# Gamma point Phonon Calculation

## write input file for ph calculation

In [9]:
# get indicies of adsorbate atoms
# add one because indexing for ph.x starts at 1, not zero
ntd = where(array(FFPt.get_chemical_symbols()) != 'Pt')[0]+1

In [10]:
ph_input(path+pre+'.in', pre, ntd, 'Pt')

## Vibrational Frequency Analysis

In [11]:
# frequencies in units of THz
freqs = ph_out(path+pre+'.out')

In [12]:
# extract frequencies for adsorbate bond vibrations
# convert frequencies to Hz from output in THz
fv = freqs[144:] * 10**12

# Constants

In [13]:
# Planck's constant
h = 4.136e-15 # eV/Hz
# temperature for analysis
T = 298.15 # K
# pressure for analysis, assume atmospheric
P = 101325 # Pa
# Boltzmann constant
k = 8.617e-5 # eV/K

# Calculate thermodynamic properties assuming ideal gas

In [14]:
# convert frequencies to energy with Planck's constant
vib_es = fv * h # eV

In [15]:
thermo = HarmonicThermo(vib_es, potentialenergy=energy, ignore_imag_modes=True)

## Get Zero Point Energy in eV

In [16]:
zpe = thermo.get_ZPE_correction()
zpe

1.58490344342

## Get entropic contributions assuming T = 298.15 K and P = 101325 Pa

In [17]:
S = thermo.get_entropy(T)

Entropy components at T = 298.15 K:
                           S               T*S
S_harm             0.0026000 eV/K        0.775 eV
-------------------------------------------------
S                  0.0026000 eV/K        0.775 eV


In [18]:
TS = T*S
TS

0.7751877967540864

## Get Helmholtz energy
Value output includes DFT potential energy and ZPE so these must be subtracted
if pV is assumed to be negligible in H = U + PV, then G = F + PV.

In [19]:
F = thermo.get_helmholtz_energy(T)

Internal energy components at T = 298.15 K:
E_pot             -44460.287 eV
E_ZPE                  1.585 eV
Cv_harm (0->T)         0.312 eV
-------------------------------
U                 -44458.390 eV

Entropy components at T = 298.15 K:
                           S               T*S
S_harm             0.0026000 eV/K        0.775 eV
-------------------------------------------------
S                  0.0026000 eV/K        0.775 eV

Free energy components at T = 298.15 K:
    U     -44458.390 eV
 -T*S         -0.775 eV
-----------------------
    F     -44459.165 eV


## Calculate Enthalpic contribution from Helmholtz energy
U = F + TS \
Cv = U - E_ZPE - E_pot

In [20]:
U = F + TS
Cv = U - zpe - energy
Cv

0.31173561362811597

## Calculate Gibbs free energy and total Gibbs free energy correction
Gibbs free energy is the Helmholtz energy and a pressure*volume term \
1 eV = 160.22e9 Pa x Å^3

In [21]:
V = FFPt.get_volume() # Å^3
V

2611.11963488235

In [22]:
PV = P*V/160.22e9
PV

0.0016513025652506186

In [23]:
G = F + PV
G

-44459.163467615195

In [24]:
Gcorr = G - energy
Gcorr

1.1231025628585485

# Summary

In [25]:
print(f'the ZPE Contribution is {zpe:.2f} eV')
print(f'the Entropic Contribution is {TS:.2f} eV')
print(f'the Enthalpy Contriibution is {Cv:.2f} eV')
print(f'the Gibbs free energy is {G:.2f} eV with a total correction of {Gcorr:.2f} eV')

the ZPE Contribution is 1.58 eV
the Entropic Contribution is 0.78 eV
the Enthalpy Contriibution is 0.31 eV
the Gibbs free energy is -44459.16 eV with a total correction of 1.12 eV
